# Visualization

In [ ]:
import xarray as xr
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

###########################################################################################################################
# Import file from the following link: https://zenodo.org/communities/ecnu_lghap/records?q=&l=list&p=1&s=10&sort=newest 
###########################################################################################################################

# === Step 1: Load and coarsen dataset ===
file_path = '/Users/saurabh/Desktop/foresaerosol_new/LGHAP.Global_PM25.D001.A20190201.nc'
ds = xr.open_dataset(file_path)
ds_india = ds.sel(lat=slice(8, 38), lon=slice(68, 98))
coarse_ds = ds_india.coarsen(lat=10, lon=10, boundary='trim').mean()
pm25 = coarse_ds['PM25'].transpose('lat', 'lon')
df_pm25 = pm25.to_dataframe().reset_index()


df_pm25['lat'] = df_pm25['lat'].astype(float).round(2)
df_pm25['lon'] = df_pm25['lon'].astype(float).round(2)

# === Step 2: Filter grid points within shapefile boundary ===
# Load shapefile
gdf_boundary = gpd.read_file("/Users/saurabh/Desktop/foresaerosol_new/shapefiles/center_shp/center.shp")

# Ensure CRS is EPSG:4326
if gdf_boundary.crs is None:
    gdf_boundary.set_crs("EPSG:4326", inplace=True)
else:
    gdf_boundary = gdf_boundary.to_crs("EPSG:4326")

# Load grid points and convert to GeoDataFrame
df_grids = pd.read_csv("/Users/saurabh/Desktop/foresaerosol_new/center_study_area/forest_fire_processing/grids_10km_india_box.csv")
geometry = [Point(xy) for xy in zip(df_grids['lon'], df_grids['lat'])]
gdf_points = gpd.GeoDataFrame(df_grids, geometry=geometry, crs="EPSG:4326")

# Spatial join: keep only points within the shapefile boundary
gdf_inside = gpd.sjoin(gdf_points, gdf_boundary, predicate='within', how='inner')
study_grids = gdf_inside.drop(columns=['geometry', 'index_right'])

study_grids['lat'] = study_grids['lat'].astype(float).round(2)
study_grids['lon'] = study_grids['lon'].astype(float).round(2)

# === Step 3: Merge PM2.5 with filtered grids ===
merged_df = pd.merge(df_pm25, study_grids, on=['lat', 'lon'])

# Keep only needed columns
filtered_df = merged_df[['lat', 'lon', 'PM25']]

# Save
filtered_df.to_csv('filtered_pm25_data.csv', index=False)

# === Step 4: Visualization ===
plt.figure(figsize=(10, 6))
scatter = plt.scatter(
    filtered_df['lon'], filtered_df['lat'],
    c=filtered_df['PM25'], cmap='inferno', s=30, edgecolor='k'
)
plt.colorbar(scatter, label='PM2.5 Concentration (µg/m³)')
plt.title('PM2.5 Concentration Over Study Area')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.tight_layout()
plt.show()


# Monthly

In [ ]:
import os
import xarray as xr
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import re
from tqdm import tqdm

###########################################################################################################################
# Import files from the following link: https://zenodo.org/communities/ecnu_lghap/records?q=&l=list&p=1&s=10&sort=newest 
# Save the file of the months required in the input_root directory.
###########################################################################################################################

# Paths
input_root = 'raw_files'
output_root = 'monthly_output'
os.makedirs(output_root, exist_ok=True)

# Load shapefile once. ############ Change the path to the shapefile as per requirement ##########
shapefile_path = "center_shp/center.shp"
gdf_boundary = gpd.read_file(shapefile_path)
if gdf_boundary.crs is None:
    gdf_boundary.set_crs("EPSG:4326", inplace=True)
else:
    gdf_boundary = gdf_boundary.to_crs("EPSG:4326")

# Load grid points once
grids_path = "/Users/saurabh/Desktop/projects/foresaerosol_new/center_study_area/forest_fire_processing/grids_10km_india_box.csv"
df_grids = pd.read_csv(grids_path)
geometry = [Point(xy) for xy in zip(df_grids['lon'], df_grids['lat'])]
gdf_points = gpd.GeoDataFrame(df_grids, geometry=geometry, crs="EPSG:4326")

# Filter grid points within shapefile boundary once
gdf_inside = gpd.sjoin(gdf_points, gdf_boundary, predicate='within', how='inner')
study_grids = gdf_inside.drop(columns=['geometry', 'index_right'])
study_grids['lat'] = study_grids['lat'].astype(float).round(2)
study_grids['lon'] = study_grids['lon'].astype(float).round(2)

# Define India bounding box
lat_slice = slice(8, 38)
lon_slice = slice(68, 98)

# Process all monthly folders
for monthly_folder in tqdm(sorted(os.listdir(input_root))):
    monthly_path = os.path.join(input_root, monthly_folder)
    if not os.path.isdir(monthly_path):
        continue

    all_dfs = []

    for nc_file in sorted(os.listdir(monthly_path)):
        if not nc_file.endswith('.nc'):
            continue

        file_path = os.path.join(monthly_path, nc_file)

        # Extract date from filename
        match = re.search(r'A(\d{8})', nc_file)
        if match:
            date_str = match.group(1)
            date_obj = datetime.strptime(date_str, '%Y%m%d')
        else:
            continue

        try:
            # Step 1: Load and coarsen dataset
            ds = xr.open_dataset(file_path)
            ds_india = ds.sel(lat=lat_slice, lon=lon_slice)
            coarse_ds = ds_india.coarsen(lat=10, lon=10, boundary='trim').mean()
            pm25 = coarse_ds['PM25'].transpose('lat', 'lon')
            df_pm25 = pm25.to_dataframe().reset_index()
            df_pm25['lat'] = df_pm25['lat'].astype(float).round(2)
            df_pm25['lon'] = df_pm25['lon'].astype(float).round(2)

            # Step 2: Merge PM2.5 with filtered grids
            merged_df = pd.merge(df_pm25, study_grids, on=['lat', 'lon'])

            # Add date column
            merged_df.insert(0, 'date', date_obj)

            # Keep only needed columns
            filtered_df = merged_df[['date', 'lat', 'lon', 'PM25']]
            all_dfs.append(filtered_df)

        except Exception as e:
            print(f"Error processing {file_path}: {e}")

    if all_dfs:
        monthly_df = pd.concat(all_dfs, ignore_index=True)
        out_filename = f"{monthly_folder}.csv"
        out_path = os.path.join(output_root, out_filename)
        monthly_df.to_csv(out_path, index=False)
        print(f"Saved {out_path}")


In [ ]:
import os
import pandas as pd
from tqdm import tqdm

# Paths
output_root = 'monthly_output'

combined_output_path = 'combined_pm25_data.csv'

# List all CSV files in the output directory
csv_files = sorted([f for f in os.listdir(output_root) if f.endswith('.csv')])

# Read and concatenate all monthly files
all_monthly_dfs = []

for file in tqdm(csv_files):
    file_path = os.path.join(output_root, file)
    try:
        df = pd.read_csv(file_path)
        all_monthly_dfs.append(df)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")

# Combine into one DataFrame
if all_monthly_dfs:
    combined_df = pd.concat(all_monthly_dfs, ignore_index=True)
    combined_df.to_csv(combined_output_path, index=False)
    print(f"Combined file saved at: {combined_output_path}")
else:
    print("No data found to combine.")
